In [1]:
from utils.dataloader import H5Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import classification_report
from tqdm import tqdm
from utils.dataloader import H5Dataset

In [2]:
num_classes = 6
k_folds = 10



# Caminho para os arquivos h5
h5_files = ['h5_files/GunshotForensic_feature.h5',
            'h5_files/HEAR_Test_feature.h5',
            'h5_files/MIVIA_test_feature.h5',
            'h5_files/MIVIA_train_feature.h5',
            'h5_files/SESA_feature.h5',
            'h5_files/vsd_clipped_features.h5'
]

# Carregando os dataloaders
dataset = H5Dataset(h5_files, exclude_classes=[6], n_splits=k_folds)
dataloaders = dataset.get_k_fold_data_loaders(batch_size=32,
                                               num_workers=4)


In [3]:
from torchvision.models import ResNet101_Weights


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Ajustando a ResNet101
model = models.resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # ajustar a última camada para 6 classes
model.conv1 = nn.Conv2d(1,
                        64,
                        kernel_size=(7, 7),
                        stride=(2, 2),
                        padding=(3, 3),
                        bias=False)  # ajustar a primeira camada para 1 canal
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Configurando o TensorBoard
writer = SummaryWriter()

cuda


In [4]:
# Loop de treinamento
for fold, (train_loader, val_loader) in enumerate(dataloaders):
    print(f'Starting fold {fold + 1}/{len(dataloaders)}')
    for epoch in range(10):  # número de épocas
        print(f'Starting epoch {epoch + 1}/10')

        # Treinamento
        model.train()
        running_loss_train = 0.0
        running_corrects_train = 0
        for i, (inputs, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
            inputs = inputs.unsqueeze(1).to(device)  # Adiciona um canal extra
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects_train += torch.sum(preds == labels.data)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss_train += loss.item() * inputs.size(0)

        epoch_loss_train = running_loss_train / len(train_loader.dataset)
        epoch_acc_train = running_corrects_train.double() / len(train_loader.dataset)

        # Validação
        model.eval()
        running_loss_val = 0.0
        running_corrects_val = 0
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs = inputs.unsqueeze(1).to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                running_corrects_val += torch.sum(preds == labels.data)
                loss = criterion(outputs, labels)

                running_loss_val += loss.item() * inputs.size(0)

                all_labels.extend(labels.data.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        epoch_loss_val = running_loss_val / len(val_loader.dataset)
        epoch_acc_val = running_corrects_val.double() / len(val_loader.dataset)

        report = classification_report(all_labels, all_preds, output_dict=True)

        print(f'Train Loss: {epoch_loss_train:.4f} Train Acc: {epoch_acc_train:.4f}')
        print(f'Validation Loss: {epoch_loss_val:.4f} Validation Acc: {epoch_acc_val:.4f}')

        f1_scores = {}
        precisions = {}
        recalls = {}
        for class_index in range(num_classes):
            f1_scores[f'class_{class_index}'] = report[str(class_index)]['f1-score']
            precisions[f'class_{class_index}'] = report[str(class_index)]['precision']
            recalls[f'class_{class_index}'] = report[str(class_index)]['recall']

        writer.add_scalars('F1 scores', f1_scores, epoch)
        writer.add_scalars('Precisions', precisions, epoch)
        writer.add_scalars('Recalls', recalls, epoch)

        writer.add_scalar('training loss', epoch_loss_train, epoch)
        writer.add_scalar('training accuracy', epoch_acc_train, epoch)
        writer.add_scalar('validation loss', epoch_loss_val, epoch)
        writer.add_scalar('validation accuracy', epoch_acc_val, epoch)

    print(f'Finished fold {fold + 1}/{len(dataloaders)}')
    
# Salvando o modelo treinado
torch.save(model.state_dict(), 'models/model_resnet101.pth')

# Fechando o TensorBoard writer
writer.close()

print('Finished Training')


Starting fold 1/10
Starting epoch 1/10


100%|██████████| 113/113 [00:57<00:00,  1.98it/s]


Train Loss: 0.6400 Train Acc: 0.7087
Validation Loss: 1.4701 Validation Acc: 0.5461
Starting epoch 2/10


100%|██████████| 113/113 [00:56<00:00,  1.99it/s]


Train Loss: 0.5559 Train Acc: 0.7502
Validation Loss: 1.4246 Validation Acc: 0.5605
Starting epoch 3/10


...

Train Loss: 0.0172 Train Acc: 0.9907
Validation Loss: 0.0127 Validation Acc: 0.9956
Starting epoch 9/10


100%|██████████| 113/113 [00:56<00:00,  1.98it/s]


Train Loss: 0.0173 Train Acc: 0.9910
Validation Loss: 0.0101 Validation Acc: 0.9972
Starting epoch 10/10


100%|██████████| 113/113 [00:56<00:00,  1.98it/s]


Train Loss: 0.0160 Train Acc: 0.9916
Validation Loss: 0.0109 Validation Acc: 0.9969
Finished fold 10/10
Finished Training


In [6]:
for fold, (train_loader, val_loader) in enumerate(dataloaders):
    print(f'Processing fold {fold + 1}/{len(dataloaders)}')

    # Carregando o modelo treinado
    model.load_state_dict(torch.load(f'models/model_resnet101.pth'))
    model.to(device)
    model.eval()

    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.data.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    report = classification_report(all_labels, all_preds)
    print(report)

print('Finished Processing')

Processing fold 1/10


100%|██████████| 113/113 [01:04<00:00,  1.74it/s]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2084
           1       1.00      0.71      0.83       324
           2       1.00      1.00      1.00       500
           3       0.77      1.00      0.87       315
           4       1.00      1.00      1.00        65
           5       1.00      1.00      1.00       316

    accuracy                           0.97      3604
   macro avg       0.96      0.95      0.95      3604
weighted avg       0.98      0.97      0.97      3604

Processing fold 2/10


100%|██████████| 113/113 [01:21<00:00,  1.39it/s]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2084
           1       1.00      1.00      1.00       324
           2       1.00      1.00      1.00       500
           3       1.00      1.00      1.00       315
           4       1.00      1.00      1.00        65
           5       1.00      1.00      1.00       316

    accuracy                           1.00      3604
   macro avg       1.00      1.00      1.00      3604
weighted avg       1.00      1.00      1.00      3604

Processing fold 3/10


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2084
           1       0.82      1.00      0.90       324
           2       1.00      1.00      1.00       500
           3       1.00      0.78      0.88       314
           4       1.00      1.00      1.00        65
           5       1.00      1.00      1.00       317

    accuracy                           0.98      3604
   macro avg       0.97      0.96      0.96      3604
weighted avg       0.98      0.98      0.98      3604

Processing fold 9/10


100%|██████████| 113/113 [01:01<00:00,  1.84it/s]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2084
           1       0.85      1.00      0.92       324
           2       1.00      1.00      1.00       500
           3       1.00      0.83      0.91       314
           4       1.00      1.00      1.00        65
           5       1.00      1.00      1.00       316

    accuracy                           0.98      3603
   macro avg       0.98      0.97      0.97      3603
weighted avg       0.99      0.98      0.98      3603

Processing fold 10/10


100%|██████████| 113/113 [01:14<00:00,  1.52it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2084
           1       0.99      1.00      1.00       324
           2       1.00      1.00      1.00       500
           3       1.00      0.98      0.99       314
           4       1.00      1.00      1.00        65
           5       0.99      1.00      1.00       316

    accuracy                           1.00      3603
   macro avg       1.00      1.00      1.00      3603
weighted avg       1.00      1.00      1.00      3603

Finished Processing
